In [55]:
import pandas as pd
from ortools.sat.python import cp_model
import itertools
import copy

A Survivor League (or Survivor Pool) in NFL is a type of elimination contest where participants pick one team each week that they believe will win their game. The goal is to remain in the pool as long as possible. Here’s how it typically works:

1. One Pick per Week: Each participant selects one NFL team to win their game in a given week.
2. Win or Eliminate: If the selected team wins, the participant survives and moves on to the next week. If the team loses (or ties, depending on the pool rules), the participant is eliminated.
3. No Repeats: Once a team is picked, it can’t be chosen again for the rest of the season. This adds strategy, as participants need to consider future weeks when choosing which team to use.
4. Last One Standing: The competition continues until only one person remains, who is then crowned the winner. If multiple participants make it through the entire season, tiebreakers are usually implemented, such as the total score of games in the final week.

Survivor leagues are popular because of their simplicity and the strategic depth required to balance short-term gains and long-term survival.

This problem looks at the 2024 NFL schedule and considers weeks 2 - 18. To get the highest probability of winning every game, I used [this link](https://www.numberfire.com/nfl/survivor-pool-matrix) that tracks each team's probability of winning per week. Next, I scale the probabilities by a factor of 10 to ensure the values are integers allowing Google's OR-Tools to implement them in our model. Finally, we are looking to maximize the overall probability of winning throughout the 18 weeks.

In [56]:
# Pulling probability table
url = 'https://www.numberfire.com/nfl/survivor-pool-matrix'
dfs = pd.read_html(url)

# Used later
dfs_temp = copy.copy(dfs[1])
dfs_temp.insert(0,'Team', dfs[0])


# Dataframe preprocessing
for i in dfs[1].columns:
    for j in range(len(dfs[1][i])):
        dfs[1].loc[j,i] = "".join(itertools.takewhile(lambda x: x!="%", dfs[1][i][j]));
        if dfs[1].loc[j,i] != 'BYE':
            dfs[1].loc[j,i] = int(float(dfs[1].loc[j,i])*10)

df = pd.DataFrame(dfs[1])
df.insert(0,'Team', dfs[0])

for i in range(len(df['Team'])):
    df['Team'][i] = df['Team'][i][:-3]
    dfs_temp['Team'][i] = dfs_temp['Team'][i][:-3]

df.set_index('Team', inplace=True)
dfs_temp.set_index('Team', inplace=True)


C:\Users\faisa\AppData\Local\Temp\ipykernel_5020\3630572808.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Team'][i] = df['Team'][i][:-3]
C:\Users\faisa\AppData\Local\Temp\ipykernel_5020\3630572808.py:22: FutureWarning: ChainedAssi

# Problem formulation
Let $x_{i,j}$ be a binary decision variable defined as:
$
x_{i,j} = \begin{cases} 
1 & \text{if team } i \text{ is assigned to week } j \\
0 & \text{otherwise}
\end{cases}
$
where $i$ indexes the teams and $j$ indexes the weeks.

Maximize the overall probability of winning games throughout the 18 weeks. This is represented by: \
\
$
\max \quad \sum_{i \in \text{Teams}} \sum_{j \in \text{Weeks}} x_{i,j} \cdot \text{prob}_{i,j}
$
\
where $\text{prob}_{i,j}$ is the probability of team $i$ winning if assigned to week $j$, and $\text{prob}_{i,j}$ is obtained from the dataframe.

1. $\textbf{Teams chosen at most once per week:}$ For each team $i$:
\
$
\sum_{j \in \text{Weeks}} x_{i,j} \leq 1
$
2. $\textbf{One team per week:}$ For each week $j$:
\
$
\sum_{i \in \text{Teams}} x_{i,j} = 1
$

3. $\textbf{Handling the "BYE" cases:}$ If $\text{prob}_{i,j} = \text{BYE}$, then $x_{i,j}$ does not contribute to the objective function.




## Mathematical Model

\begin{array}{lrcllr}
    \text{maximize }  & z & = & \sum_{i}^{Teams} \sum_{j}^{Weeks} {x_{i,j}} prob_{ij} & & (1) \\
    \text{subject to} &  \sum_{i}^{Teams}  x_{i,j} & = & 1 & \forall j \in Weeks & (2) \\
                      &  \sum_{j}^{Weeks}  x_{i,j} & \leq & 1 & \forall i \in Teams & (3) \\
                      & x_{ij} &\in &\{0,1\} & \forall (i,j) \in (Teams \times Weeks)  & (4) \\
    \end{array}

In [57]:
# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

Teams = df.index
Weeks = df.columns

x = {(i,j): model.NewBoolVar(f'x_{i}_{j}') for i in Teams for j in Weeks}

# Maximizing the overall probability of winning games throughout the 18 weeks
model.Maximize(sum(x[i,j]*df.loc[i,j] for i in Teams for j in Weeks if df.loc[i,j] != 'BYE'))

# One team per week
for j in Weeks:
    model.AddExactlyOne([x[i,j] for i in Teams if df.loc[i,j] != 'BYE'])

# Teams chosen at most once 
for i in Teams:
    model.AddAtMostOne([x[i,j] for j in Weeks if df.loc[i,j] != 'BYE'])

status = solver.Solve(model)
print(f'Status = {solver.StatusName(status)}')
print(f'Probability of winning out: {solver.ObjectiveValue()/180}')

Status = OPTIMAL
Probability of winning out: 78.87222222222222


In [58]:
[f'Week {j}: {i} defeat {dfs_temp.loc[i][j][-3:]}' for j in Weeks for i in Teams if solver.value(x[i,j]) == 1]

['Week 2: Detroit Lions  defeating  TB',
 'Week 3: Las Vegas Raiders defeating CAR',
 'Week 4: San Francisco 49ers defeating  NE',
 'Week 5: Chicago Bears  defeating CAR',
 'Week 6: Baltimore Ravens  defeating WSH',
 'Week 7: Jacksonville Jaguars  defeating  NE',
 'Week 8: Pittsburgh Steelers  defeating NYG',
 'Week 9: Cincinnati Bengals  defeating  LV',
 'Week 10: Kansas City Chiefs defeating DEN',
 'Week 11: Miami Dolphins  defeating  LV',
 'Week 12: Houston Texans  defeating TEN',
 'Week 13: Dallas Cowboys  defeating NYG',
 'Week 14: Philadelphia Eagles  defeating CAR',
 'Week 15: New Orleans Saints defeating WSH',
 'Week 16: Buffalo Bills  defeating  NE',
 'Week 17: Tampa Bay Buccaneers defeating CAR',
 'Week 18: Atlanta Falcons  defeating CAR']

In [59]:
# sol = np.reshape([solver.value(x[i,j]) if df.loc[i,j] != 'BYE' else 'BYE' for i in df.index for j in df.columns], (len(df.index),len(df.columns)))

# res = pd.DataFrame.from_records(sol)
# res.columns = list(range(2,19))
# res.index = list(df.index)
# res